<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Generate-some-random-data" data-toc-modified-id="Generate-some-random-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Generate some random data</a></span></li><li><span><a href="#Solve-the-carbon-management-problem" data-toc-modified-id="Solve-the-carbon-management-problem-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Solve the carbon management problem</a></span></li><li><span><a href="#Test-the-dynamic-network" data-toc-modified-id="Test-the-dynamic-network-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Test the dynamic network</a></span><ul class="toc-item"><li><span><a href="#One-single-timestep,-with-no-battery-capacity" data-toc-modified-id="One-single-timestep,-with-no-battery-capacity-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>One single timestep, with no battery capacity</a></span></li><li><span><a href="#Several-timesteps,-with-no-battery-capacity" data-toc-modified-id="Several-timesteps,-with-no-battery-capacity-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Several timesteps, with no battery capacity</a></span></li><li><span><a href="#Several-timesteps,-with-battery-capacity" data-toc-modified-id="Several-timesteps,-with-battery-capacity-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Several timesteps, with battery capacity</a></span></li><li><span><a href="#Several-timesteps,-with-battery-capacity-and-varying-demand" data-toc-modified-id="Several-timesteps,-with-battery-capacity-and-varying-demand-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Several timesteps, with battery capacity and varying demand</a></span></li></ul></li><li><span><a href="#Plot-results" data-toc-modified-id="Plot-results-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot results</a></span><ul class="toc-item"><li><span><a href="#Plot-demand,-generation,-and-LMPs" data-toc-modified-id="Plot-demand,-generation,-and-LMPs-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Plot demand, generation, and LMPs</a></span></li><li><span><a href="#Plot-carbon-LMPs" data-toc-modified-id="Plot-carbon-LMPs-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Plot carbon LMPs</a></span></li></ul></li><li><span><a href="#Implicit-diff" data-toc-modified-id="Implicit-diff-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Implicit diff</a></span></li><li><span><a href="#Compute-MEFs-via-regression" data-toc-modified-id="Compute-MEFs-via-regression-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Compute MEFs via regression</a></span></li></ul></div>

In [ ]:
using Pkg; Pkg.activate()

using Plots
using Convex, ECOS  # Convex modeling and solver
using Distributions, Random  # Seeds and sampling
using LightGraphs  # Generating nice random graphs
using GraphPlot, Colors  # For plotting
using SparseArrays

OPT = () -> ECOS.Optimizer(verbose=false)

In [ ]:
using Revise
using CarbonNetworks

# Generate some random data

In [ ]:
Random.seed!(2)
n = 4
l = 4

# Make graph
G = watts_strogatz(n, 3, 0.2)

# Convert to incidence matrix
A = incidence_matrix(G, oriented=true)
m = size(A, 2)

# Create generator-node mapping
node_map = vcat(collect(1:n), rand(1:n, l-n))
B = spzeros(n, l)
for (gen, node) in enumerate(node_map)
    B[node, gen] = 1
end

# Generate carbon costs
cq = zeros(l)
cl = rand(Exponential(2), l)

# Generate demands
d = rand(Bernoulli(0.8), n) .* rand(Gamma(3.0, 3.0), n)

# Generate generation and flow capacities
gmax = rand(Gamma(4.0, 3.0), l) + (B'd)  # This is just to make the problem easier
pmax = rand(Gamma(1.0, 0.1), m)

# Plot network
c_node = (B*cl) ./ (B*ones(l))
nodesize = maximum(d) .+ d
nodefillc = weighted_color_mean.((c_node / maximum(c_node)).^0.5, colorant"red", colorant"green")
edgelinewidth = (pmax / maximum(pmax)).^0.5

Random.seed!(0)  # I put a seed here so the spring layout doesn't change
plt = gplot(G, nodesize=nodesize, nodefillc=nodefillc, edgelinewidth=edgelinewidth)

# Solve the carbon management problem

In [ ]:
cnet = PowerNetwork(cq, cl, pmax, gmax, A, B);

In [ ]:
carbon_min = PowerManagementProblem(cnet, d)
solve!(carbon_min, OPT, verbose=true);

In [ ]:
carbon_min.p.value

In [ ]:
carbon_min.g.value

In [ ]:
carbon_min.problem.optval

In [ ]:
# checking continuity

A*carbon_min.p.value - B*carbon_min.g.value + d

# Test the dynamic network

## One single timestep, with no battery capacity

This should give the same result as above

In [ ]:
T = 1

cq_dyn = [cq for _ in 1:T]
cl_dyn = [cl for _ in 1:T]
pmax_dyn = [pmax for _ in 1:T]
gmax_dyn = [gmax for _ in 1:T]
d_dyn = [d for _ in 1:T]

P = rand(Exponential(2), n)
C = rand(Exponential(2), n)
C = zeros(n)
;

In [ ]:
dnet = DynamicPowerNetwork(
    cq_dyn, cl_dyn, pmax_dyn, gmax_dyn, A, B, P, C
)
dmin = DynamicPowerManagementProblem(dnet, d_dyn);

In [ ]:
solve!(dmin, OPT, verbose=true);

In [ ]:
dmin.p[1].value

In [ ]:
dmin.g[1].value

In [ ]:
dmin.s[1].value

## Several timesteps, with no battery capacity

Again, this should be the same as $T$ uncoupled problems. 

In [ ]:
T = 5

cq_dyn = [cq for _ in 1:T]
cl_dyn = [cl for _ in 1:T]
pmax_dyn = [pmax for _ in 1:T]
gmax_dyn = [gmax for _ in 1:T]
d_dyn = [d for _ in 1:T]

P = rand(Exponential(2), n)
C = rand(Exponential(2), n)
C = zeros(n)
;

In [ ]:
dnet = DynamicPowerNetwork(
    cq_dyn, cl_dyn, pmax_dyn, gmax_dyn, A, B, P, C
)
dmin = DynamicPowerManagementProblem(dnet, d_dyn);

In [ ]:
solve!(dmin, OPT, verbose=true);

In [ ]:
for t in 1:T
    @show dmin.p[t].value
    @show dmin.g[t].value
    @show dmin.s[1].value
end

In [ ]:
s_vals = zeros(n, T)
g_vals = zeros(n, T)
for t in 1:T
    s_vals[:, t] = dmin.s[t].value
    g_vals[:, t] = dmin.g[t].value
end

In [ ]:
plot(s_vals')

In [ ]:
plot(g_vals')

## Several timesteps, with battery capacity

Again, this should be the same as $T$ uncoupled problems. 

In [ ]:
T = 5

cq_dyn = [cq for _ in 1:T]
cl_dyn = [cl for _ in 1:T]
pmax_dyn = [pmax for _ in 1:T]
gmax_dyn = [gmax for _ in 1:T]
d_dyn = [d for _ in 1:T]

P = rand(Exponential(2), n)
C = rand(Exponential(2), n)
;

In [ ]:
dnet = DynamicPowerNetwork(
    cq_dyn, cl_dyn, pmax_dyn, gmax_dyn, A, B, P, C
)
dmin = DynamicPowerManagementProblem(dnet, d_dyn);

In [ ]:
solve!(dmin, OPT, verbose=true);

In [ ]:
s_vals = zeros(n, T)
g_vals = zeros(n, T)
for t in 1:T
    s_vals[:, t] = dmin.s[t].value
    g_vals[:, t] = dmin.g[t].value
end

In [ ]:
plot(s_vals')

In [ ]:
plot(g_vals')

## Several timesteps, with battery capacity and varying demand

Again, this should be the same as $T$ uncoupled problems. 

In [ ]:
T = 5

cq_dyn = [cq for _ in 1:T]
cl_dyn = [cl for _ in 1:T]
pmax_dyn = [pmax for _ in 1:T]
gmax_dyn = [gmax for _ in 1:T]
d_dyn = [rand(Bernoulli(0.8), n) .* rand(Gamma(2.0, 2.0), n) for _ in 1:T]

P = rand(Exponential(2), n)
C = rand(Exponential(2), n)
;

In [ ]:
dnet = DynamicPowerNetwork(
    cq_dyn, cl_dyn, pmax_dyn, gmax_dyn, A, B, P, C
)
dmin = DynamicPowerManagementProblem(dnet, d_dyn);

In [ ]:
solve!(dmin, OPT, verbose=true);

In [ ]:
s_vals = zeros(n, T)
g_vals = zeros(n, T)
for t in 1:T
    s_vals[:, t] = dmin.s[t].value
    g_vals[:, t] = dmin.g[t].value
end

In [ ]:
plot(s_vals')

In [ ]:
plot(g_vals')

----

----

# Plot results

`GraphPlot` is only for making simple plots. To do more advanced things (like the plot in the carbon accounting paper), we'll need to draw the graph manually.

For now, I'm going to make some simple diagnostic plots.

## Plot demand, generation, and LMPs

Here, the LMPs are carbon LMPs.

In [ ]:
λ = get_lmps(carbon_min)  # <-- LMPs

# Fancy colors
gradient = cgrad(:inferno)
lmp_colors = [get(gradient, λi/maximum(λ)) for λi in λ]


plt1 = bar(d, label=nothing, c=:blue, ylabel="demand")
plt2 = bar(evaluate(carbon_min.g), label=nothing, c=:green, ylabel="generation")
plt3 = sticks(λ, label=nothing, ylabel="carbon LMP", xlabel="node")
scatter!(plt3, λ, c=lmp_colors, label=nothing)

plot(plt1, plt2, plt3, layout=(3, 1))

## Plot carbon LMPs

Brighter is more carbon intensive. At the bright nodes (orange and yellow), increasing / decreasing demand has the greatest effect on total emissions.

In [ ]:
Random.seed!(0)  # <-- For the node layout consistency
plt = gplot(G, nodesize=nodesize, nodefillc=lmp_colors, edgelinewidth=edgelinewidth)

# Implicit diff

In [ ]:
using CarbonNetworks: kkt, flatten_variables

Random.seed!(4)

fq = 3 .+ rand(Exponential(3), l)  # generate monetary costs
fl = 3 .+ rand(Exponential(3), l)

# Solve cost minimization problem
net = PowerNetwork(fq, fl, pmax, gmax, A, B)
opf = PowerManagementProblem(net, d)
solve!(opf, OPT, verbose=true)

kkt_resid = kkt(flatten_variables(opf), net, d)
@show norm(kkt_resid)

In [ ]:
sum(abs, evaluate(opf.p)) / sum(d)

In [ ]:
∇C = zeros(kkt_dims(n, m, l))
∇C[1:l] .= cl

mefs = sensitivity_demand(opf, ∇C, net, d)

theme(:default, legend=false)
plt1a = bar(evaluate(carbon_min.g), title="carbon min generation", color=:green)
plt1b = bar(get_lmps(carbon_min), title="carbon min LMP", color=:yellow)
plt2a = bar(evaluate(opf.g), title="opf generation", color=:green)
plt2b = bar(mefs, title="marginal emission factors", color=:yellow)
plot(plt1a, plt1b, plt2a, plt2b, layout=(2, 2), size=(700, 400))

# Compute MEFs via regression

In [ ]:
using CarbonNetworks: solve_pmp

Random.seed!(14)

num_cases = 1000
ϵ = 0.05

d_hidden = rand(Uniform(0.3, 0.7), n) .* (B*gmax)

flows = []
cases = []
for _ in 1:num_cases
    d_hidden *= rand(Uniform(0.95, 1.05))
    d_obs = d_hidden + ϵ*randn(n)
    
    g, pmp, _ = solve_pmp(fq, fl, d_obs, pmax, gmax, A, B)
    if Int(pmp.problem.status) ∉ [1, 7]
        @show pmp.problem.status
    end
    
    push!(cases, (d=d_obs, g=g, pmp=pmp))
    push!(flows, sum(abs, evaluate(pmp.p)) / sum(g))
end

In [ ]:
Random.seed!(41)
inds = sample(1:n, 5, replace=false)

theme(:default, label=nothing, lw=4)
plot([d[i] for (d, g) in cases, i in inds], labels=inds')

In [ ]:
E = [B * (cl .* g) for (d, g) in cases]

Δd = hcat([cases[t].d - cases[t-1].d for t in 2:num_cases]...)
ΔE = hcat([E[t] - E[t-1] for t in 2:num_cases]...);

In [ ]:
theme(:default, label=nothing)

plts = []
for i in 1:n
    μi = Δd[i, :] \ ΔE[i, :]
    ΔE_est = Δd[i, :] * μi
    correlation = cor(ΔE_est, ΔE[i, :])

    #@show correlation
    #@show μi

    μi_diffs = []

    for case in cases[1:100]
        params = (fq, fl, case.d, pmax, gmax, A)
        ∇C = zeros(kkt_dims(n, m, l))
        ∇C[1:l] .= cl

        mefs = sensitivity_demand(case.pmp, ∇C, net, case.d)
        push!(μi_diffs, mefs[i])
    end
    μi_diff = mean(μi_diffs)

    #@show μi_diff
    @show i, minimum(μi_diffs), maximum(μi_diffs)
    # println()

    x_coords = Δd[i, :]
    
    label = (i > 1) ? nothing : "mean mef"
    label2 = (i > 1) ? nothing : "max mef"
    
    sim_mean = μi_diff*x_coords
    sim_max = maximum(μi_diffs)*x_coords
    sim_min = minimum(μi_diffs)*x_coords

    plt = scatter(x_coords, ΔE[i, :], xlabel="Δd", ylabel="ΔE", ms=2, title="$i")
    plot!(plt, x_coords, ΔE_est, lw=2) #, label="r2=$(round(correlation, digits=2))", legend=:topleft)
    plot!(plt, x_coords, sim_mean, lw=2, label=label, legend=:topleft, color=:Green)
    plot!(plt, x_coords, maximum(μi_diffs)*x_coords, lw=2, label=label2, color=:Cyan, ls=:dot)
    plot!(plt, x_coords, minimum(μi_diffs)*x_coords, lw=2, label=label2, color=:Cyan, ls=:dot)
    push!(plts, plt)
end

plot(plts..., layout=(2, 5), size=(900, 450))

In [ ]:
edges_15 = findall(x -> x != 0, A[15, :])
edges_16 = findall(x -> x != 0, A[16, :])
@show edges_15 edges_16

#heatmap(Matrix(neighbors), c=:grays, yflip=true)
plot()
bar(pmax)
bar!(evaluate(cases[100].pmp.p), xticks=([30 34 40], [30, 34, 40]))
